In [16]:
import time

class Perfo:

    def timeExecution(functionCalled, *args):

        start_time = time.time()
        function(args)
        return start_time - time.time()

class FordFulkerson:

    """
        Solve Ford-Fulkerson Algorithm

        Attributes:
            graph ([Int]) : adjency matrix
            source_node (int) : Start node
            sink_node (int) : End node 
            n (int) : number of node in graph
            visited_token (int) : indicator to know last time a node was visited 
            visited [int] : list with the node visited (0 if never visited by default)
    
    """

    def __init__(self, graph, source_node, sink_node):
        self.graph = graph 
        self.source_node = source_node  
        self.sink_node = sink_node 
        self.n = len(graph) 
        self.visited_token = 1
        self.visited = [0] * self.n


    def dfs(self, node, sink, flow):

        """
            Depth First Search

            Params:
                node (int) : source node
                sink(int) : end node
                flow (int) : number of path existed between node and sink

            Returns:
                flow(int)
        
        """
        
        if node == sink:
            return flow
        
        self.visited[node] = self.visited_token

        for i in range(self.n):

            if self.visited[i] != self.visited_token and  self.graph[node][i] > 0:

                min_flow = min(flow, self.graph[node][i]) 
                dfs_flow = self.dfs(i, sink, min_flow)

                if dfs_flow:
                    self.graph[node][i] -= dfs_flow
                    self.graph[i][node] += dfs_flow

                    return dfs_flow
                
        return 0

    def max_flow(self):

        """
            Ford-Fulkerson Algorithm 
        
        """
        
        max_flow = 0
        while True:

            flow = self.dfs(self.source_node, self.sink_node, float('Inf'))
            self.visited_token += 1

            if flow == 0:  
                return max_flow

            max_flow += flow


graph_flux = [
    [0, 10, 0, 10],  
    [0, 0, 4, 2],    
    [0, 0, 0, 10],   
    [0, 0, 0, 0],    
]
source = 0
sink = 3

ff = FordFulkerson(graph_flux, source, sink)

print(ff.max_flow())




Le flux maximal est: 16
